# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils_experiments import *

import tensorflow as tf

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Activation, Dense, Flatten
from keras import initializers
from tensorflow.keras.metrics import R2Score

In [2]:
# Set the plot style
plt.style.use('seaborn-v0_8-bright')  # Or any other style you prefer

# Update rcParams for font, figure size, and other settings
plt.rcParams.update({
    'font.family': 'serif',        # Use serif fonts
    'font.serif': ['Times New Roman'],  # Specifically Times New Roman
    'font.size': 12,              # Font size for all text
    'axes.labelsize': 14,         # Axis label size
    'axes.titlesize': 16,         # Title size
    'xtick.labelsize': 12,        # x-axis tick label size
    'ytick.labelsize': 12,        # y-axis tick label size
    'legend.fontsize': 12,        # Legend font size
    'figure.figsize': [6, 4],     # Aspect ratio
    'savefig.dpi': 300,           # High-resolution output for saving
})

# Load Data

In [3]:
porosity = np.load('data/data_vashisth/porosity_data.npy')
seismic = np.load('data/data_vashisth/seismic_data.npy')

In [ ]:
porosity.shape, seismic.shape

In [5]:
dt=7e-4
end_time=0.172
time_upscaled = np.arange(0, end_time, dt)
diffDepth = 1
depth = np.arange(1,porosity.shape[1]+1)*diffDepth

In [ ]:
fig, ax = plt.subplots(1,6, figsize=(15,5))

ax[0].plot(np.squeeze(porosity[1,:]), depth)
ax[0].invert_yaxis()
ax[0].set_ylabel('Depth [m]')
ax[0].set_xlabel('Porosity [v/v]')


ax[1].plot(np.squeeze(seismic[1,:]), time_upscaled)
ax[1].invert_yaxis()
ax[1].set_ylabel('Time [s]')
ax[1].set_xlabel('Seismic')

ax[2].plot(np.squeeze(porosity[10,:]), depth)
ax[2].invert_yaxis()
ax[2].set_ylabel('Depth [m]')
ax[2].set_xlabel('Porosity [v/v]')


ax[3].plot(np.squeeze(seismic[10,:]), time_upscaled)
ax[3].invert_yaxis()
ax[3].set_ylabel('Time [s]')
ax[3].set_xlabel('Seismic')

ax[4].plot(np.squeeze(porosity[100,:]), depth)
ax[4].invert_yaxis()
ax[4].set_ylabel('Depth [m]')
ax[4].set_xlabel('Porosity [v/v]')


ax[5].plot(np.squeeze(seismic[100,:]), time_upscaled)
ax[5].invert_yaxis()
ax[5].set_ylabel('Time [s]')
ax[5].set_xlabel('Seismic')


#fig.suptitle('Porosidad vs Sísmica')
fig.tight_layout()
fig.savefig('plots/vashisth_example.eps', format='eps')

# Data Wrangling
## Depth to time domain conversion

In [ ]:
Kg = 0.06
Kw = 2.61
RHOw = 1.01
RHOg = 0.18 
diffP = 16.5 
RHOs = 2.65
PhiC = 0.4 
Coordination = 14 
Fudge = 1
diffDepth = 20
vclay=tf.cast(0,tf.float64)
sw=tf.cast(1,tf.float64)
Kf = 1/(sw/Kw + (1 - sw)/Kg)
RHOf = sw*RHOw + (1 - sw) * RHOg


In [8]:
def SoftSandNew(Kf,RHOf,Phi,Quartz,Clay,Feldspar,Limestone,Dolomite,Pressure,PhiC,Coordination,Fudge):
    # Uncemented sand rock physics model (modified to implement automatic-differentiation)
    #               INPUTS
    #
    #Kf				Fluid bulk modulus
    #RHOf			Fluid density
    #Phi			Porosity (fraction)
    #Quartz, etc.	Volume mineral content in solid phase (fraction)
    #Pressure		Effective pressure (MPa)
    #PhiC			Critical porosity ~0.4
    #Coordination	Coordination number ~6
    #Fudge			Reduction in shear stiffness -- 1 by default
    #
    #a is the contact radius between two spheres at input pressure

    #Balancing mineralogy
    Dolomite=1-(Quartz+Clay+Feldspar+Limestone)
    
    #Constants for clay
    RHOClay=2.65 
    KClay=21 
    GClay=7
    #Solid-phase elastic moduli
    KsV=Quartz*36.6+Clay*KClay+Limestone*76.8+Dolomite*94.9+Feldspar*75.6
    KsR=1/(Quartz/36.6+Clay/KClay+Limestone/76.8+Dolomite/94.9+Feldspar/75.6)
    Ks=0.5*(KsV+KsR)
    
    GsV=Quartz*45+Clay*GClay+Limestone*32+Dolomite*45+Feldspar*25.6
    GsR=1/(Quartz/45+Clay/GClay+Limestone/32+Dolomite/45+Feldspar/25.6)
    Gs=0.5*(GsV+GsR)
    Ms=Ks+(4/3)*Gs
    NUs=0.5*(Ms/Gs-2)/(Ms/Gs-1)
    
    #Solid-phase density
    RHOs=Quartz*2.65+Clay*RHOClay+Limestone*2.71+Dolomite*2.87+Feldspar*2.63
    
    #ILLITE
    #RHOs=2.58; Ms=RHOs.*4.32.*4.32; Gs=RHOs.*2.54.*2.54; NUs=0.5*(Ms./Gs-2)./(Ms./Gs-1); Ks=Ms-(4/3)*Gs;
    
    P=Pressure/1000
    C=Coordination
    
    #================== Dry Rock ====================
    #Effective K and G at PhiC
    #Hertz contact radius (R=1)
    a=tf.pow(((3*3.14*(1-NUs)/(2*C*(1-PhiC)*Gs))*P),(1/3))
    #Normal contact stiffness
    SN=4*a*Gs/(1-NUs)
    #Shear contact stiffness
    ST=8*a*Gs/(2-NUs)
    #Shear contact stiffness FUDGED (reduced)
    ST=ST/Fudge
    
    #Bulk modulus
    Khat=C*(1.-PhiC)*SN/(12*3.14)
    #Shear modulus
    Ghat=C*(1.-PhiC)*(SN+1.5*ST)/(20*3.14)
    
    #Effective bulk and shear moduli at porosity Phi<=PhiC
    KDry1=1/((Phi/PhiC)/(Khat+4*Ghat/3)+((PhiC-Phi)/PhiC)/(Ks+4*Ghat/3))-4*Ghat/3
    ZZ1=(Ghat/6)*(9*Khat+8*Ghat)/(Khat+2*Ghat)
    GDry1=1/((Phi/PhiC)/(Ghat+ZZ1)+((PhiC-Phi)/PhiC)/(Gs+ZZ1))-ZZ1
    MDry1 = KDry1+(4/3)*GDry1
    NuDry1=0.5*(MDry1/GDry1-2)/(MDry1/GDry1-1)
    
    #Effective bulk and shear moduli at porosity Phi>PhiC
    KDry2 = 1/(((1-Phi)/(1-PhiC))/(Khat+4*Ghat/3)+((Phi-PhiC)/(1-PhiC))/(4*Ghat/3))-4*Ghat/3
    ZZ2 = (Ghat/6)*(9*Khat+8*Ghat)/(Khat+2*Ghat)
    GDry2 = 1/(((1-Phi)/(1-PhiC))/(Ghat+ZZ2)+((Phi-PhiC)/(1-PhiC))/(ZZ2))-ZZ2
    MDry2 = KDry2+(4/3)*GDry2
    NuDry2=0.5*(MDry2/GDry2-2)/(MDry2/GDry2-1)
    
    MDry=MDry1*(tf.cast((Phi<=PhiC), tf.float64))+MDry2*(tf.cast((Phi>PhiC), tf.float64)) 
    GDry=GDry1*(tf.cast((Phi<=PhiC), tf.float64))+GDry2*(tf.cast((Phi>PhiC), tf.float64))
    NuDry=NuDry1*(tf.cast((Phi<=PhiC), tf.float64))+NuDry2*(tf.cast((Phi>PhiC), tf.float64))
    KDry=MDry-(4/3)*GDry
    
    # ================== Saturated Rock ============
    KSat=Ks*(Phi*KDry-(1+Phi)*Kf*KDry/Ks+Kf)/((1-Phi)*Kf+Phi*Ks-Kf*KDry/Ks)
    MSat=KSat+(4/3)*GDry
    RHOB=(1-Phi)*RHOs+Phi*RHOf
    Vp=tf.pow((MSat/RHOB),0.5)
    Vs=tf.pow((GDry/RHOB),0.5)
    Ip=Vp*RHOB
    PR=0.5*(MSat/GDry-2)/(MSat/GDry-1)
    
    return Vp,Vs,RHOB

In [9]:
def DecoderLinearInterpLayer(x,xp,fp):
    i = tf.clip_by_value(tf.searchsorted(xp, x, side='right'), 1, xp.shape[1] - 1)

    fp1=tf.gather(fp,indices=i-1, batch_dims=1)
    fp2=tf.gather(fp,indices=i, batch_dims=1)
    xp1=tf.gather(xp,indices=i-1, batch_dims=1)
    xp2=tf.gather(xp,indices=i, batch_dims=1)

    f = (fp1 *  (xp2 - x) + fp2 * (x - xp1)) / (xp2 - xp1)

    left=fp[0,0]
    right = fp[-1,-1]

    ff = tf.where(x < tf.ones_like(x)*tf.expand_dims(xp[:,0],axis=1), left, f)
    fff = tf.where(x > tf.ones_like(x)*tf.expand_dims(xp[:,-1],axis=1), tf.ones_like(x)*tf.expand_dims(fp[:,-1],axis=1), ff)
    return fff

In [10]:
Vp_final,Vs_final,RHO_final=SoftSandNew(Kf,RHOf,porosity,(1 - vclay)*0.8,vclay, (1 - vclay)*0.2,0,0,diffP, PhiC, Coordination, Fudge)

depth_diff = np.mean(np.diff(depth.T))
time0=0.0/(Vp_final[:,0])
time1 = 2*tf.cumsum(depth_diff/(Vp_final*1e3),axis=1)
time_final = tf.concat([tf.expand_dims(time0, axis=1),time1],axis=1)
time = time_final[:,0:-1]

time_new=tf.ones_like(time, dtype=tf.float64)
for i in range(len(time_upscaled)-time.shape[1]):
    time_new=tf.concat([time_new,tf.expand_dims(time0+1.0,axis=1)],axis=1)
pred_t=time_new*time_upscaled

Phi_time=DecoderLinearInterpLayer(pred_t,time,porosity)
RHO_time=DecoderLinearInterpLayer(pred_t,time,RHO_final)


## First visualization of data

In [ ]:
seed_og = 318 #np.random.randint(0,2500)
trace_porosity = Phi_time[seed_og,:]
trace_seismic = seismic[seed_og,:]
seed_og

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,5))

ax[0].plot(np.squeeze(trace_porosity), time_upscaled)
ax[0].invert_yaxis()
ax[0].set_ylabel('Time [s]')
ax[0].set_xlabel('Porosity [v/v]')


ax[1].plot(np.squeeze(trace_seismic), time_upscaled)
ax[1].invert_yaxis()
ax[1].set_ylabel('Time [s]')
ax[1].set_xlabel('Seismic')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(trace_seismic, trace_porosity, edgecolor='black', linewidth=0.5)
ax.set_xlabel('Seismic [v/v]')
ax.set_ylabel('Porosity')

fig.savefig('plots/vashisth_crossplot_correlation.eps', format='eps')

In [14]:
phi_time_numpy = Phi_time.numpy()

## Normalization of data

In [15]:
def min_max_scale(x, min, max):

  x_std = (x - min) / (max - min)
  x_scaled = x_std * 2 - 1
  return x_scaled

def inverse_min_max_scale(x, min, max):

  x_normalized = (x + 1) / 2
  x_unscaled = x_normalized * (max - min) + min
  return x_unscaled

### Porosity

In [16]:
phi_max=np.max(phi_time_numpy)
phi_min=np.min(phi_time_numpy)

In [17]:
phi_scaled = min_max_scale(phi_time_numpy, min= phi_min, max=phi_max)
phi_unscaled = inverse_min_max_scale(phi_scaled, min= phi_min, max=phi_max)

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(phi_time_numpy.ravel(), bins=30, edgecolor='black', color='#43a2ca')
# Add titles and labels
plt.title('Distribution of Porosity', fontsize=16, fontweight='bold')
plt.xlabel('Porosity [v/v]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add grid lines
plt.grid(axis='y', linestyle='--', alpha=0.9)

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a text box annotation
textstr = 'Mean: {:.2f}\nStd Dev: {:.2f}'.format(np.mean(phi_time_numpy.ravel()), np.std(phi_time_numpy.ravel()))
plt.gca().text(0.95, 0.95, textstr, transform=plt.gca().transAxes,
               fontsize=12, verticalalignment='top',
               bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='lightgray'))

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(phi_scaled.ravel(), bins=30, edgecolor='black', color='#43a2ca')
# Add titles and labels
plt.title('Distribution of Normalized Porosity', fontsize=16, fontweight='bold')
plt.xlabel('Porosity [unitless]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add grid lines
plt.grid(axis='y', linestyle='--', alpha=0.9)

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a text box annotation
textstr = 'Mean: {:.2f}\nStd Dev: {:.2f}'.format(np.mean(phi_scaled.ravel()), np.std(phi_scaled.ravel()))
plt.gca().text(0.95, 0.95, textstr, transform=plt.gca().transAxes,
               fontsize=12, verticalalignment='top',
               bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='lightgray'))

plt.show()

### Seismic

In [20]:
# Normalizing the input and output data  
seis_normalized = (seismic - np.min(seismic))/(np.max(seismic)-np.min(seismic))

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(seis_normalized.ravel(), bins=30, edgecolor='black', color='#43a2ca')
# Add titles and labels
plt.title('Distribution of Normalized Seismic', fontsize=16, fontweight='bold')
plt.xlabel('Seismic [unitless]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add grid lines
plt.grid(axis='y', linestyle='--', alpha=0.9)

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a text box annotation
textstr = 'Mean: {:.2f}\nStd Dev: {:.2f}'.format(np.mean(seis_normalized.ravel()), np.std(seis_normalized.ravel()))
plt.gca().text(0.95, 0.95, textstr, transform=plt.gca().transAxes,
               fontsize=12, verticalalignment='top',
               bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='lightgray'))

# Show plot
plt.savefig('./plots/normalized_seismic_distribution.eps', format='eps')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(seismic.ravel(), bins=30, edgecolor='black', color='#43a2ca')
# Add titles and labels
plt.title('Distribution of Seismic', fontsize=16, fontweight='bold')
plt.xlabel('Seismic [unitless]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add grid lines
plt.grid(axis='y', linestyle='--', alpha=0.9)

# Customize ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a text box annotation
textstr = 'Mean: {:.2f}\nStd Dev: {:.2f}'.format(np.mean(seismic.ravel()), np.std(seismic.ravel()))
plt.gca().text(0.95, 0.95, textstr, transform=plt.gca().transAxes,
               fontsize=12, verticalalignment='top',
               bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='lightgray'))

# Show plot
plt.savefig('./plots/seismic_distribution.eps', format='eps')
plt.show()

In [ ]:
np.min(seis_normalized), np.max(seis_normalized)

In [ ]:
np.min(phi_scaled), np.max(phi_scaled)

In [ ]:
seismic.shape, phi_scaled.shape

## Traind and test split

In [26]:
X_train_norm = seis_normalized[:1750, :]
y_train_norm = phi_scaled[:1750, :]
X_test_norm = seis_normalized[1750:, :]
y_test_norm = phi_scaled[1750:, :]

In [27]:
X_train = seismic[:1750, :]
y_train = phi_time_numpy[:1750, :]
X_test = seismic[1750:, :]
y_test = phi_time_numpy[1750:, :]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Reshape

This reshape is done to train algorithms such as DecisionTree, XGBoost and MLP using all the data

In [29]:
X_train_reshaped = X_train.reshape(-1,1)
X_test_reshaped = X_test.reshape(-1,1)

y_train_reshaped = y_train.reshape(-1,1)
y_test_reshaped = y_test.reshape(-1,1)

In [ ]:
X_train_reshaped.shape, X_test_reshaped.shape, y_train_reshaped.shape, y_test_reshaped.shape

# Machine Learning

## Linear Regression

In [31]:
lin_reg = LinearRegression().fit(trace_seismic.reshape(-1,1), trace_porosity)

In [32]:
y_pred_linear = lin_reg.predict(trace_seismic.reshape(-1,1))

In [33]:
line_x = np.array([-0.2, 0.2])
line_y = lin_reg.predict(line_x.reshape(-1,1))

### Metrics

In [ ]:
rmse_rl = np.sqrt(mean_squared_error(trace_porosity, y_pred_linear))
MAE_rl = mean_absolute_error(trace_porosity, y_pred_linear)
trace_porosity = trace_porosity.numpy()
r2_rl = r2_score(trace_porosity, y_pred_linear)

print("RMSE : % f" %(rmse_rl))
print("MAE : % f" %(MAE_rl))
print("R2 : % f" %(r2_rl))

In [ ]:
print(f'y = {lin_reg.coef_[0]:3.3}x + {lin_reg.intercept_:3.3}')

### Plot

In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(trace_seismic, trace_porosity, edgecolor='black', linewidth=0.5)
ax.plot(line_x, line_y, color='red', linewidth=2, label='Linear Regression')
# Add textbox with R2
textstr = rf"$R^2 = {r2_rl:.3f}$"
props = dict(boxstyle='round', facecolor='white', alpha=0.8)

ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

# Add legend
legend = ax.legend(loc='upper right', fontsize='small')
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(1)

ax.set_xlabel('Seismic ')
ax.set_ylabel('Porosity [v/v]')

fig.savefig('plots/vashisth_crossplot_regression.eps', format='eps')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,5), sharey=True)

ax[0].plot(np.squeeze(trace_porosity), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_linear), time_upscaled, label='Model Porosity', linestyle='--', color='red')
ax[0].invert_yaxis()
ax[0].set_xlabel('Porosity [v/v]')
ax[0].set_ylabel('Time [s]')


ax[1].plot(np.squeeze(trace_seismic), time_upscaled)
ax[1].set_xlabel('Seismic')

ax[0].legend(loc='upper center',    bbox_to_anchor=(0.5, 1.2))
fig.tight_layout()
fig.savefig('plots/resultado_regresion_lineal_seccion.eps', format='eps')

## Quadratic Regression

In [38]:
from sklearn.preprocessing import PolynomialFeatures

In [39]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
poly_features = poly.fit_transform(trace_seismic.reshape(-1, 1))
poly_reg_model = LinearRegression()
poly_reg_model.fit(poly_features, trace_porosity)

### Metrics

In [41]:
y_predicted_quadratic = poly_reg_model.predict(poly_features)

In [ ]:
rmse_quadratic = np.sqrt(mean_squared_error(trace_porosity, y_predicted_quadratic))
MAE_quadratic = mean_absolute_error(trace_porosity, y_predicted_quadratic)
r2_quadratic = r2_score(trace_porosity, y_predicted_quadratic)

print("RMSE : % f" %(rmse_quadratic))
print("MAE : % f" %(MAE_quadratic))
print("R2 : % f" %(r2_quadratic))

### Plot

In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(trace_seismic, trace_porosity, edgecolor='black', linewidth=0.5)
ax.plot(trace_seismic, y_predicted_quadratic, color='red', linewidth=2, label='Quadratic Regression')

textstr = rf"$R^2 = {r2_quadratic:.3f}$"
props = dict(boxstyle='round', facecolor='white', alpha=0.8)

ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)


legend = ax.legend(loc='upper right', fontsize='small')
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(1)

ax.set_xlabel('Seismic ')
ax.set_ylabel('Porosity [v/v]')

ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

fig.savefig('plots/vashisth_crossplot_quadratic.eps', format='eps')

## Decision Tree

### 1 trace test

In [44]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn import tree

In [45]:
dtr = DecisionTreeRegressor()
dtr_controled = DecisionTreeRegressor(min_samples_leaf=10)

In [ ]:
dtr.fit(trace_seismic.reshape(-1, 1), trace_porosity)
dtr_controled.fit(trace_seismic.reshape(-1, 1), trace_porosity)

In [47]:
y_pred_tree_dtr_train = dtr.predict(trace_seismic.reshape(-1, 1))

In [ ]:
rmse_dtr_train = np.sqrt(mean_squared_error(trace_porosity, y_pred_tree_dtr_train))
MAE_dtr_train = mean_absolute_error(trace_porosity, y_pred_tree_dtr_train)
r2_dtr_train = r2_score(trace_porosity, y_pred_tree_dtr_train)

print("RMSE : % f" %(rmse_dtr_train))
print("MAE : % f" %(MAE_dtr_train))
print("R2 : % f" %(r2_dtr_train))

#### Metrics - Blind Test

In [ ]:
seed_test = 1409 #np.random.randint(0,2500)
trace_porosity_test = Phi_time[seed_test,:]
trace_seismic_test = seismic[seed_test,:]
seed_test

In [50]:
y_pred_tree_test = dtr.predict(trace_seismic_test.reshape(-1, 1))
y_pred_tree_test_controled = dtr_controled.predict(trace_seismic_test.reshape(-1, 1))

In [ ]:
rmse_dtr_test_1well = np.sqrt(mean_squared_error(trace_porosity_test, y_pred_tree_test))
MAE_dtr_test_1well = mean_absolute_error(trace_porosity_test, y_pred_tree_test)
trace_porosity_test=trace_porosity_test.numpy()
r2_dtr_test_1well = r2_score(trace_porosity_test, y_pred_tree_test)

print("RMSE : % f" %(rmse_dtr_test_1well))
print("MAE : % f" %(MAE_dtr_test_1well))
print("R2 : % f" %(r2_dtr_test_1well))

In [ ]:
rmse_dtr_test_1well_depth5 = np.sqrt(mean_squared_error(trace_porosity_test, y_pred_tree_test_controled))
mae_dtr_test_1well_depth5 = mean_absolute_error(trace_porosity_test, y_pred_tree_test_controled)

r2_dtr_test_1well_depth5 = r2_score(trace_porosity_test, y_pred_tree_test_controled)

print("RMSE : % f" %(rmse_dtr_test_1well_depth5))
print("MAE : % f" %(mae_dtr_test_1well_depth5))
print("R2 : % f" %(r2_dtr_test_1well_depth5))

#### Plot

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True, sharex=True)

ax[0].plot(np.squeeze(trace_porosity), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_tree_dtr_train), time_upscaled, label='Model Porosity', linestyle='--', color='red')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_title('Train Data')


ax[1].plot(np.squeeze(trace_porosity_test), time_upscaled, label='Real Porosity', color='black')
ax[1].plot(np.squeeze(y_pred_tree_test), time_upscaled, label='Model Porosity', linestyle='--', color='red')
ax[1].set_title('Test Data')

fig.text(0.5, 0.07, 'Porosity [v/v]', ha='center', va='center')

ax[1].legend(loc='lower center')

fig.savefig('plots/dt_1_trace.eps', format='eps')

In [ ]:
params = dtr.get_params()
max_depth = dtr.get_depth()
n_leaves = dtr.get_n_leaves()

print("Model Parameters:", params)
print("Max depth reached:", max_depth)
print("Number of leaves:", n_leaves)

In [ ]:
# Obtener los parámetros y metadatos
params = dtr_controled.get_params()
max_depth = dtr_controled.get_depth()
n_leaves = dtr_controled.get_n_leaves()

print("Model Parameters:", params)
print("Max depth reached:", max_depth)
print("Number of leaves:", n_leaves)

In [ ]:
plt.figure(figsize=(20, 10)) 
plot_tree(dtr, filled=True, fontsize=10)
plt.savefig('plots/decision_tree.eps', format='eps')
plt.show()

### All dataset test

In [ ]:
dtr_full = DecisionTreeRegressor()
dtr_full.fit(X_train_reshaped, y_train_reshaped)

In [58]:
y_pred_train = dtr_full.predict(X_train_reshaped)
y_pred_test = dtr_full.predict(X_test_reshaped)

#### Metrics

In [ ]:
rmse_dtr_train = np.sqrt(mean_squared_error(y_train_reshaped, y_pred_train))
MAE_dtr_train = mean_absolute_error(y_train_reshaped, y_pred_train)
r2_dtr_train = r2_score(y_train_reshaped, y_pred_train)

print("RMSE : % f" %(rmse_dtr_train))
print("MAE : % f" %(MAE_dtr_train))
print("R2 : % f" %(r2_dtr_train))

In [ ]:
rmse_dtr_test = np.sqrt(mean_squared_error(y_test_reshaped, y_pred_test))
MAE_dtr_test = mean_absolute_error(y_test_reshaped, y_pred_test)
r2_dtr_test = r2_score(y_test_reshaped, y_pred_test)

print("RMSE : % f" %(rmse_dtr_test))
print("MAE : % f" %(MAE_dtr_test))
print("R2 : % f" %(r2_dtr_test))

#### Plot

In [61]:
y_pred_train_reshaped = y_pred_train.reshape(1750, 246)
y_pred_test_reshaped = y_pred_test.reshape(750, 246)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True, sharex=True)

ax[0].set_title('Train Data')
ax[0].plot(np.squeeze(y_train[354,:]), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_train_reshaped[354,:]), time_upscaled, label='Model Porosity', linestyle='--', color='red')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()

ax[1].set_title('Test Data')
ax[1].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Real Porosity', color='black')
ax[1].plot(np.squeeze(y_pred_test_reshaped[256,:]), time_upscaled, label='Model Porosity', linestyle='--', color='red')

fig.text(0.5, 0.07, 'Porosity [v/v]', ha='center', va='center')

ax[1].legend(loc='lower left',bbox_to_anchor=(1, 0.5))

fig.savefig('plots/dt_results.eps', format='eps')

In [ ]:
params = dtr_full.get_params()
max_depth = dtr_full.get_depth()
n_leaves = dtr_full.get_n_leaves()

print("Model parameters:", params)
print("Max depth reached:", max_depth)
print("Number of leaves:", n_leaves)

## XGBoost

In [64]:
import xgboost as xg 
from sklearn.model_selection import RandomizedSearchCV

In [65]:
xgb_r = xg.XGBRegressor(seed=123)

param_dist = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 300, 500, 1000],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'booster': ['gbtree', 'dart']
}

random_search = RandomizedSearchCV(estimator=xgb_r, 
                                   param_distributions=param_dist, 
                                   scoring='neg_mean_squared_error', 
                                   n_iter=10, 
                                   cv=3, 
                                   verbose=1)



In [ ]:
xgb_r.fit(X_train_reshaped, y_train_reshaped)

In [67]:
pred_xgboost = xgb_r.predict(X_test_reshaped)

### Metrics

In [ ]:
rmse_xgboost = np.sqrt(mean_squared_error(y_test_reshaped, pred_xgboost)) 
mae_xgboost = mean_absolute_error(y_test_reshaped, pred_xgboost)
r2_xgboost = r2_score(y_test_reshaped, pred_xgboost)

print("RMSE : % f" %(rmse_xgboost))
print("MAE : % f" %(mae_xgboost))
print("R2 : % f" %(r2_xgboost))

### Plot

In [69]:
y_pred_xgboost_reshaped = pred_xgboost.reshape(750, 246)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True)

ax[0].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_xgboost_reshaped[256,:]), time_upscaled, label='Model Porosity', color='red', linestyle='--')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_xlabel('Porosity [v/v]')

ax[1].plot(np.squeeze(X_test[500,:]), time_upscaled)
ax[1].set_xlabel('Seismic')

ax[0].legend(loc='lower right')
plt.plot()

## MLP

In [71]:
epochs=100

In [ ]:
X_train.shape, y_train.shape

### Architecture

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(100, activation='tanh',input_shape=(246,)),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(246, activation='tanh'),
])
model.summary()

In [74]:
model.compile(optimizer='adam',
                    loss='mse',
                    metrics=[R2Score()])

In [75]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('models/first_nn.weights.h5', 
                                                monitor='r2_score', 
                                                mode='max', 
                                                verbose=1, 
                                                save_weights_only=True)

callback_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='r2_score', 
                                                   factor=0.5, 
                                                   patience=10, 
                                                   min_lr=0.0001)

### Training

In [ ]:
history_nn = model.fit(X_train, 
                       y_train, 
                       epochs=epochs,
                       batch_size=10,
                        callbacks=[checkpoint, callback_lr] )

model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
history_nn.history.keys()

In [ ]:
epochs_list = [i for i in range(epochs)]

train_acc = history_nn.history['r2_score']
train_loss = history_nn.history['loss']
train_lr = history_nn.history['learning_rate']

fig, ax = plt.subplots(1,3)
fig.set_size_inches(16,7)

ax[0].plot(epochs_list, train_acc, 'go-', label='accuracy-train')
ax[0].set_title('Accuracy train')
ax[0].legend()
ax[0].set_xlabel('epochs')
ax[0].set_ylabel('accuracy')
ax[0].set_ylim(0,1)

ax[1].plot(epochs_list, train_loss, 'go-', label='loss-train')
ax[1].set_title('Loss train')
ax[1].legend()
ax[1].set_xlabel('epochs')
ax[1].set_ylabel('loss')

ax[2].plot(epochs_list, train_lr, 'go-', label='lr-train')
ax[2].set_title('Learning Rate train')
ax[2].legend()
ax[2].set_xlabel('epochs')
ax[2].set_ylabel('Learning Rate')

fig.savefig('./plots/training.eps', format='eps')

plt.show()

### Metrics

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
y_pred_nn = model.predict(X_test)

In [ ]:
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
MAE_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print("RMSE : % f" %(rmse_nn))
print("MAE : % f" %(MAE_nn))
print("R2 : % f" %(r2_nn))

### Plot

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True)

ax[0].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_nn[256,:]), time_upscaled, label='Model Porosity', color='red', linestyle='--')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_xlabel('Porosity [v/v]')

ax[1].plot(np.squeeze(X_test[500,:]), time_upscaled)
ax[1].set_xlabel('Seismic')

ax[0].legend(loc='lower right')

fig.savefig('plots/resultado_cnn_seccion.eps', format='eps')

## MLP (Normalized Data)

In [ ]:
X_train_norm.min(), X_train_norm.max(), y_train_norm.min(), y_train_norm.max()

### Training

In [ ]:
model2_mlp = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(246,)),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(246, activation='tanh'),
])
model2_mlp.summary()

In [85]:
model2_mlp.compile(optimizer='adam',
                    loss='mse',
                    metrics=[R2Score()])

In [ ]:
history2_nn = model2_mlp.fit(X_train_norm, 
                       y_train_norm, 
                       epochs=epochs,
                       batch_size=10,
                        callbacks=[checkpoint, callback_lr] )

model2_mlp.evaluate( X_test_norm,  y_test_norm, verbose=2)

In [ ]:
epochs_list = [i for i in range(epochs)]

train_acc = history2_nn.history['r2_score']
train_loss = history2_nn.history['loss']
train_lr = history_nn.history['learning_rate']

fig, ax = plt.subplots(1,3)
fig.set_size_inches(16,7)

ax[0].plot(epochs_list, train_acc, 'go-', label='accuracy-train')
ax[0].set_title('Accuracy train')
ax[0].legend()
ax[0].set_xlabel('epochs')
ax[0].set_ylabel('accuracy')
ax[0].set_ylim(0,1)

ax[1].plot(epochs_list, train_loss, 'go-', label='loss-train')
ax[1].set_title('Loss train')
ax[1].legend()
ax[1].set_xlabel('epochs')
ax[1].set_ylabel('loss')

ax[2].plot(epochs_list, train_lr, 'go-', label='lr-train')
ax[2].set_title('Learning Rate train')
ax[2].legend()
ax[2].set_xlabel('epochs')
ax[2].set_ylabel('Learning Rate')

fig.savefig('./plots/training.eps', format='eps')

plt.show()

### Metrics

In [ ]:
loss, accuracy = model2_mlp.evaluate(X_test_norm, y_test_norm)

In [ ]:
y_pred_nn_norm = model2_mlp.predict(X_test_norm)

In [ ]:
rmse_nn_norm = np.sqrt(mean_squared_error(y_test_norm, y_pred_nn_norm))
MAE_nn_norm  = mean_absolute_error(y_test_norm, y_pred_nn_norm)
r2_nn_norm  = r2_score(y_test_norm, y_pred_nn_norm)

print("RMSE : % f" %(rmse_nn_norm))
print("MAE : % f" %(MAE_nn_norm))
print("R2 : % f" %(r2_nn_norm))

In [91]:
y_pred_nn_norm = inverse_min_max_scale(y_pred_nn_norm, phi_min, phi_max)

### Plot

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True)

ax[0].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_nn_norm[256,:]), time_upscaled, label='Model Porosity', color='red', linestyle='--')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_xlabel('Porosity [v/v]')


ax[1].plot(np.squeeze(X_test[500,:]), time_upscaled)
ax[1].set_xlabel('Seismic')


#fig.suptitle('Comparacion entre porosidad modelada y real', y=0.9)
ax[0].legend(loc='lower right')
#fig.tight_layout()
fig.savefig('plots/resultado_cnn_seccion.eps', format='eps')

## Convolutional Neural Netowrk (CNN)

In [93]:
X_train_final_cnn = np.expand_dims(X_train_norm, axis=-1)
X_test_final_cnn = np.expand_dims(X_test_norm, axis=-1)

In [ ]:
X_train_final_cnn.shape, X_test_final_cnn.shape

In [ ]:
y_train_norm.shape, y_test_norm.shape

In [ ]:
X_train_final_cnn.min(), X_train_final_cnn.max(), y_train_norm.min(), y_train_norm.max()

### Architecture

In [ ]:
given_seed = 13
def naive_model():
    """El input shape de tal forma sería entonces (1, 246) ya que son trazas sismicas, 2000 de ellas"""

    model_phi = tf.keras.models.Sequential()
    
    model_phi.add(Conv1D(input_shape=(246,1), filters=32, kernel_size=2, strides=2, padding='same',kernel_initializer=initializers.he_uniform(seed=given_seed), bias_initializer='zeros')) 
    model_phi.add(Activation('relu'))
    
    model_phi.add(Conv1D(filters=64, kernel_size=7, strides=2, padding='same',kernel_initializer=initializers.he_uniform(seed=given_seed), bias_initializer='zeros')) 
    model_phi.add(Activation('relu'))
    
    model_phi.add(Conv1D(filters=128, kernel_size=15, strides=2, padding='same',kernel_initializer=initializers.he_uniform(seed=given_seed), bias_initializer='zeros')) 
    model_phi.add(Activation('relu'))
    
    model_phi.add(Conv1D(filters=256, kernel_size=30, strides=2, padding='same',kernel_initializer=initializers.he_uniform(seed=given_seed), bias_initializer='zeros')) 
    model_phi.add(Activation('relu'))
    
    model_phi.add(tf.keras.layers.Dropout(rate=0.3))
    

    model_phi.add(Flatten())
 
    
    model_phi.add(Dense(units=246,kernel_initializer=initializers.he_uniform(seed=given_seed), bias_initializer='zeros'))
    model_phi.add(Activation('tanh'))

    return model_phi
model_phi = naive_model()
model_phi.summary()

In [ ]:
tf.keras.utils.plot_model(model_phi, show_shapes=True, dpi=72, to_file='model.png')

In [99]:
optimizer = tf.keras.optimizers.Adam()

model_phi.compile(optimizer=optimizer,
                    loss='mse',
                    metrics=[R2Score()])

In [100]:
#callback_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint('models/first_cnn.weights.h5', 
                                                monitor='r2_nicolas', 
                                                mode='max', 
                                                verbose=1, 
                                                save_weights_only=True)

callback_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='r2_nicolas', 
                                                   factor=0.5, 
                                                   patience=10, 
                                                   min_lr=0.0001)

### Training

In [ ]:
history_early = model_phi.fit(x=X_train_final_cnn, 
                                y=y_train_norm,
                                epochs=epochs,
                                batch_size=10,
                                callbacks=[checkpoint, callback_lr]                         
                                )

### Metrics

In [ ]:
epochs_list = [i for i in range(epochs)]

train_acc = history_early.history['r2_score']
train_loss = history_early.history['loss']
train_lr = history_early.history['learning_rate']

fig, ax = plt.subplots(1,3)
fig.set_size_inches(16,7)

ax[0].plot(epochs_list, train_acc, 'go-', label='accuracy-train')
ax[0].set_title('Accuracy train')
ax[0].legend()
ax[0].set_xlabel('epochs')
ax[0].set_ylabel('accuracy')
ax[0].set_ylim(0,1)

ax[1].plot(epochs_list, train_loss, 'go-', label='loss-train')
ax[1].set_title('Loss train')
ax[1].legend()
ax[1].set_xlabel('epochs')
ax[1].set_ylabel('loss')

ax[2].plot(epochs_list, train_lr, 'go-', label='lr-train')
ax[2].set_title('Learning Rate train')
ax[2].legend()
ax[2].set_xlabel('epochs')
ax[2].set_ylabel('Learning Rate')

fig.savefig('./plots/training.eps', format='eps')

plt.show()

In [ ]:
loss, accuracy = model_phi.evaluate(X_test_final_cnn, y_test_norm)

In [ ]:
y_pred_cnn = model_phi.predict(X_test_final_cnn)

In [ ]:
rmse_cnn = np.sqrt(mean_squared_error(y_test_norm, y_pred_cnn))
MAE_cnn = mean_absolute_error(y_test_norm, y_pred_cnn)
r2_cnn = r2_score(y_test_norm, y_pred_cnn)

print("RMSE : % f" %(rmse_cnn))
print("MAE : % f" %(MAE_cnn))
print("R2 : % f" %(r2_cnn))

In [106]:
y_pred_cnn = inverse_min_max_scale(y_pred_cnn, phi_min, phi_max)

### Plot

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,8), sharey=True)

ax[0].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Real Porosity', color='black')
ax[0].plot(np.squeeze(y_pred_cnn[256,:]), time_upscaled, label='Model Porosity', color='red', linestyle='--')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_xlabel('Porosity [v/v]')

ax[1].plot(np.squeeze(X_test[500,:]), time_upscaled)
ax[1].set_xlabel('Seismic')

ax[0].legend(loc='lower right')

fig.savefig('plots/resultado_cnn_seccion.eps', format='eps')

# Final Results

In [108]:
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.framealpha'] = 1
plt.rcParams['legend.fancybox'] = False  # Disable rounded corners
plt.rcParams['legend.borderaxespad'] = 0.5  # Adjust if needed

In [109]:
y_predict_linear = lin_reg.predict(X_test[256,:].reshape(-1,1))

poly_features_final = poly.fit_transform(X_test[256,:].reshape(-1, 1))
y_predict_quadratic = poly_reg_model.predict(poly_features)

y_predict_tree = dtr.predict(X_test[256,:].reshape(-1, 1))

In [ ]:
fig, ax = plt.subplots(1,7, figsize=(15,8), sharey=True, sharex=True)

ax[0].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[0].plot(np.squeeze(y_predict_linear), time_upscaled, label='Model Porosity', color='red', linestyle='--')
ax[0].set_ylabel('Time [s]')
ax[0].invert_yaxis()
ax[0].set_title('Linear Model', fontsize=12)

ax[1].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[1].plot(np.squeeze(y_predict_quadratic), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[1].set_title('Quadratic\nModel', fontsize=12)

ax[2].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[2].plot(np.squeeze(y_predict_tree), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[2].set_title('Decision Tree\nModel', fontsize=12)


ax[3].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[3].plot(np.squeeze(y_pred_xgboost_reshaped[256,:]), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[3].set_title('XGBoost\nModel', fontsize=12)

ax[4].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[4].plot(np.squeeze(y_pred_nn[256,:]), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[4].set_title('Multi Layer\nPercepton Model', fontsize=12)

ax[5].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[5].plot(np.squeeze(y_pred_nn_norm[256,:]), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[5].set_title('MLP Normalized\nData', fontsize=12)

ax[6].plot(np.squeeze(y_test[256,:]), time_upscaled, label='Ground Truth Porosity', color='black')
ax[6].plot(np.squeeze(y_pred_cnn[256,:]), time_upscaled, label='Predicted Porosity', color='red', linestyle='--')
ax[6].set_title('CNN Model', fontsize=12)

fig.suptitle('Machine Learning Models')
fig.text(0.5, 0.07, 'Porosity [v/v]', ha='center', va='center')

# Customize the legend
legend = ax[6].legend(loc='lower left', bbox_to_anchor=(-0.5, 0))
legend.get_frame().set_facecolor('white')  # Set background to white
legend.get_frame().set_alpha(1)  # Remove transparency
legend.get_frame().set_edgecolor('black')  # Set edge color to black
legend.get_frame().set_linewidth(1)  # Set line width to 1

plt.show()
fig.savefig('plots/compiled_result.eps', format='eps')

# Table

In [ ]:
metodo = {'Method': ['LR', 'QR', 'DT', 'DT (Full Dataset)', 'XGBoost', 'MLP', 'MLP (Normalized)','CNN'],
            'R-Squared':[r2_rl, r2_quadratic, r2_dtr_test_1well, r2_dtr_test, r2_xgboost, r2_nn, r2_nn_norm, r2_cnn],
            'MSE':[rmse_rl, rmse_quadratic, rmse_dtr_test_1well, rmse_dtr_test, rmse_xgboost, rmse_nn, rmse_nn_norm, rmse_cnn],
            'MAE':[MAE_rl, MAE_quadratic, MAE_dtr_test_1well, MAE_dtr_test, mae_xgboost, MAE_nn, MAE_nn_norm, MAE_cnn]
          }

df_ml_techniques = pd.DataFrame(data=metodo)
df_ml_techniques

In [ ]:
latex_table = df_ml_techniques.to_latex(index=False, float_format="%.3f")
print(latex_table)